In [12]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

config = SparkConf() \
                    .setAppName('Delta Lake') \
                    .setAll([('spark.executor.memory', '4G'),
                            ('spark.driver.memory', '2G'),
                            ('spark.driver.maxResultSize', '1G')]) \
                    .set("spark.jars.packages", "com.amazonaws:aws-java-sdk-s3:1.12.161,org.apache.hadoop:hadoop-aws:3.3.4,io.delta:delta-core_2.12:2.1.1") \
                    .set("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
                    .set("spark.hadoop.fs.s3a.access.key", "minio") \
                    .set("spark.hadoop.fs.s3a.secret.key", "minio123") \
                    .set("spark.hadoop.fs.s3a.path.style.access", True) \
                    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
                    .set("spark.hadoop.fs.s3a.path.style.access", "true") \
                    .set("spark.hadoop.fs.s3a.committer.name", "directory") \
                    .set("spark.hadoop.fs.s3a.committer.staging.conflict-mode", "replace") \
                    .set("spark.hadoop.fs.s3a.committer.staging.tmp.path", "/tmp/staging") \
                    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
                    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

sc = SparkContext(conf=config)
spark = SparkSession(sc)

# spark = SparkSession.builder.config(conf=config)

spark

2022-11-20 15:26:38,253 INFO server.AbstractConnector: Stopped Spark@2218bd55{HTTP/1.1, (http/1.1)}{0.0.0.0:4040}
2022-11-20 15:26:38,253 INFO ui.SparkUI: Stopped Spark web UI at http://localhost:4040
2022-11-20 15:26:38,259 INFO spark.MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
2022-11-20 15:26:38,264 INFO memory.MemoryStore: MemoryStore cleared
2022-11-20 15:26:38,264 INFO storage.BlockManager: BlockManager stopped
2022-11-20 15:26:38,264 INFO storage.BlockManagerMaster: BlockManagerMaster stopped
2022-11-20 15:26:38,264 INFO scheduler.OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
2022-11-20 15:26:38,291 INFO spark.SparkContext: Successfully stopped SparkContext
2022-11-20 15:26:38,817 INFO spark.SparkContext: Running Spark version 3.3.1
2022-11-20 15:26:38,817 INFO resource.ResourceUtils: ==============================================================
2022-11-20 15:26:38,817 INFO resource.ResourceUtils: No custom

In [10]:
from datetime import datetime

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
from pyspark.sql.functions import lit
from delta.tables import DeltaTable

In [13]:
data =  [{'id': 1, 'name': 'Alice', 'updated_at': datetime(2022, 1, 1)},
         {'id': 2, 'name': 'Braga', 'updated_at': datetime(2022, 2, 2)},
         {'id': 3, 'name': 'Steve', 'updated_at': datetime(2022, 3, 3)}]

schema = StructType([StructField('id', IntegerType(), nullable=True),
                     StructField('name', StringType(), nullable=True),
                     StructField('updated_at', DateType(), nullable=True)])

df = spark.createDataFrame(data, schema=schema)
df.toPandas()

,id,name,updated_at
0,1,Alice,2022-01-01
1,2,Braga,2022-02-02
2,3,Steve,2022-03-03


## Create Delta Table

### HDFS

In [5]:
# Create or replace partitioned table with path using DataFrame's schema and write/overwrite data to it
df.write.format("delta") \
  .mode("overwrite") \
  .option("mergeSchema", "true") \
  .option("userMetadata", "some comments") \
  .save("/delta-lake/users")

# Create table in the metastore using DataFrame's schema and write data to it
# df.write.format("delta") \
#   .mode("overwrite") \
#   .option("mergeSchema", "true") \
#   .saveAsTable("users")

### Minio (S3)

In [ ]:
df.write.format("delta") \
  .mode("overwrite") \
  .option("mergeSchema", "true") \
  .save("s3a://my-bucket//delta-lake/users")

### Create table without data

In [7]:
my_table = DeltaTable.createOrReplace(spark) \
  .addColumn("id", "INT") \
  .addColumn("firstName", "STRING") \
  .addColumn("middleName", "STRING") \
  .addColumn("lastName", "STRING", comment = "surname") \
  .addColumn("gender", "STRING") \
  .addColumn("birthDate", "TIMESTAMP") \
  .addColumn("ssn", "STRING") \
  .addColumn("salary", "INT") \
  .property("description", "table with people data") \
  .location("/delta-lake/my-table") \
  .execute()

In [23]:
# Read as delta format
my_table = DeltaTable.forPath(spark, "/delta-lake/my-table")
my_table.history().toPandas()

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,0,2022-02-20 22:04:09.548,None,None,CREATE OR REPLACE TABLE,"{'description': None, 'partitionBy': '[]', 'pr...",None,None,None,NaN,SnapshotIsolation,True,{},None,Apache-Spark/3.2.1 Delta-Lake/1.1.0


In [24]:
my_table.toDF().toPandas()

,id,firstName,middleName,lastName,gender,birthDate,ssn,salary


## Read Delta Table

### HDFS

In [6]:
# Read as delta format
spark.read.format("delta") \
     .load("/delta-lake/users") \
     .toPandas()

,id,name,updated_at
0,2,Braga,2022-02-02
1,3,Steve,2022-03-03
2,1,Alice,2022-01-01


In [7]:
spark.sql("""
    SELECT * FROM delta.`/delta-lake/users` -- query table by path
""").toPandas()

,id,name,updated_at
0,2,Braga,2022-02-02
1,3,Steve,2022-03-03
2,1,Alice,2022-01-01


In [28]:
# Read as parquet format
spark.read.format("parquet") \
     .load("/delta-lake/users") \
     .toPandas()

,id,name,updated_at
0,1,Alice,2022-01-02
1,4,Carell,2022-04-04
2,2,Braga,2022-02-02
3,3,Steve,2022-03-03
4,1,Alice,2022-01-01


### Minio (S3)

In [ ]:
spark.read.format("delta") \
     .load("s3a://my-bucket/delta-lake/users") \
     .toPandas()

# spark.read.format("parquet") \
#      .load("s3a://my-bucket//delta-lake/users") \
#      .show()

## Describe tables

In [29]:
users_table = DeltaTable.forPath(spark, "/delta-lake/users")
users_table.history().toPandas()

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,1,2022-02-20 22:11:18.529,None,None,MERGE,"{'matchedPredicates': '[{""actionType"":""update""...",None,None,None,0.0,Serializable,False,"{'numOutputRows': '2', 'numTargetRowsInserted'...",None,Apache-Spark/3.2.1 Delta-Lake/1.1.0
1,0,2022-02-20 22:03:47.787,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,NaN,Serializable,False,"{'numOutputRows': '3', 'numOutputBytes': '1880...",some comments,Apache-Spark/3.2.1 Delta-Lake/1.1.0


In [14]:
spark.sql("""
    DESCRIBE DETAIL delta.`/delta-lake/users` -- query table by path
""").toPandas()

,format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
0,delta,0b8f1ca3-c7bf-4740-89ea-678beefb4baf,None,None,hdfs://localhost:9000/delta-lake/users,2022-02-20 22:03:45.023,2022-02-20 22:03:47.787,[],2,1880,{},1,2


In [15]:
# Only if delta table saved as table
spark.sql("""
    DESCRIBE DETAIL users
""").toPandas()

,format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
0,delta,931b0889-4c4f-4f45-98aa-d64052cb5fbd,default.users,None,file:/opt/apps/spark-warehouse/users,2022-02-20 18:40:31.589,2022-02-20 22:03:55.757,[],2,1880,{},1,2


In [16]:
# Only if delta table saved as table
spark.sql("""
    SELECT * FROM users
    """).toPandas()

,id,name,updated_at
0,2,Braga,2022-02-02
1,3,Steve,2022-03-03
2,1,Alice,2022-01-01


In [17]:
# Tables in metastore
spark.sql("""
    SHOW tables
""").toPandas()

,namespace,tableName,isTemporary
0,default,users,False


## Upsert (Merge) new Data

In [115]:
users_table = DeltaTable.forPath(spark, "/delta-lake/users")

In [90]:
new_data = [{'id': 1, 'name': 'Alice', 'updated_at': datetime(2022, 1, 2)},
            {'id': 4, 'name': 'Carell', 'updated_at': datetime(2022, 4, 4)}]

new_data_df = spark.createDataFrame(new_data, schema=schema)
new_data_df.toPandas()

,id,name,updated_at
0,1,Alice,2022-01-02
1,4,Carell,2022-04-04


In [ ]:
users_table.alias("old_data") \
           .merge(source=new_data_df.alias("new_data"), condition="old_data.id = new_data.id") \
           .whenMatchedUpdate(set={
                                    "updated_at": "new_data.updated_at"
                              }) \
           .whenNotMatchedInsert(values={
                                            "id": "new_data.id",
                                            "name": "new_data.name",
                                            "updated_at": "new_data.updated_at"
                                }) \
           .execute()

users_table.toDF().toPandas()

In [91]:
users_table.alias("old_data") \
           .merge(source=new_data_df.alias("new_data"), condition="old_data.id = new_data.id") \
           .whenMatchedUpdateAll() \
           .whenNotMatchedInsertAll() \
           .execute()

users_table.toDF().toPandas()

,id,name,updated_at
0,1,Alice,2022-01-02
1,4,Carell,2022-04-04
2,2,Braga,2022-02-02
3,3,Steve,2022-03-03


### Upsert with Missing fields

In [92]:
data_missing_fields = [{"id": 5, 'name': 'Joao'}]

data_missing_fields = spark.createDataFrame(data_missing_fields, schema=schema)
data_missing_fields.toPandas()

,id,name,updated_at
0,5,Joao,None


In [93]:
users_table.alias("old_data") \
           .merge(source=data_missing_fields.alias("new_data"), condition="old_data.id = new_data.id") \
           .whenMatchedUpdateAll() \
           .whenNotMatchedInsertAll() \
           .execute()

users_table.toDF().limit(10).toPandas()

,id,name,updated_at
0,1,Alice,2022-01-02
1,4,Carell,2022-04-04
2,2,Braga,2022-02-02
3,3,Steve,2022-03-03
4,5,Joao,None


## Update Schema adding column

In [100]:
# Read as delta format
update_schema = spark.read.format("delta").load("/delta-lake/users")

In [101]:
# Add column
update_schema = update_schema.withColumn("age", lit(None).cast(StringType()))
update_schema.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- updated_at: date (nullable = true)
 |-- age: string (nullable = true)



In [102]:
update_schema.limit(10).toPandas()

,id,name,updated_at,age
0,1,Alice,2022-01-02,None
1,4,Carell,2022-04-04,None
2,2,Braga,2022-02-02,None
3,3,Steve,2022-03-03,None
4,5,Joao,None,None


In [106]:
# Merge Schema, another option is overwriteSchema
update_schema.write.format("delta") \
  .mode("overwrite") \
  .option("mergeSchema", "true") \
  .option("userMetadata", "add age column") \
  .save("/delta-lake/users")

In [119]:
DeltaTable.forPath(spark, "/delta-lake/users").history().toPandas()

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,5,2022-02-20 22:56:30.573,None,None,MERGE,"{'matchedPredicates': '[{""actionType"":""update""...",None,None,None,4.0,Serializable,False,"{'numOutputRows': '7', 'numTargetRowsInserted'...",None,Apache-Spark/3.2.1 Delta-Lake/1.1.0
1,4,2022-02-20 22:50:27.894,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,3.0,Serializable,False,"{'numOutputRows': '10', 'numOutputBytes': '243...",add age column,Apache-Spark/3.2.1 Delta-Lake/1.1.0
2,3,2022-02-20 22:48:36.639,None,None,WRITE,"{'mode': 'Append', 'partitionBy': '[]'}",None,None,None,2.0,Serializable,False,"{'numOutputRows': '5', 'numOutputBytes': '2254...",add age column,Apache-Spark/3.2.1 Delta-Lake/1.1.0
3,2,2022-02-20 22:46:07.823,None,None,MERGE,"{'matchedPredicates': '[{""actionType"":""update""...",None,None,None,1.0,Serializable,False,"{'numOutputRows': '1', 'numTargetRowsInserted'...",None,Apache-Spark/3.2.1 Delta-Lake/1.1.0
4,1,2022-02-20 22:45:53.184,None,None,MERGE,"{'matchedPredicates': '[{""actionType"":""update""...",None,None,None,0.0,Serializable,False,"{'numOutputRows': '2', 'numTargetRowsInserted'...",None,Apache-Spark/3.2.1 Delta-Lake/1.1.0
5,0,2022-02-20 22:45:34.193,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,NaN,Serializable,False,"{'numOutputRows': '3', 'numOutputBytes': '1880...",some comments,Apache-Spark/3.2.1 Delta-Lake/1.1.0


In [118]:
DeltaTable.forPath(spark, "/delta-lake/users").toDF().toPandas()

,id,name,updated_at,age
0,1,Alice,2022-01-02,30
1,1,Alice,2022-01-02,30
2,2,Braga,2022-02-02,None
3,3,Steve,2022-03-03,None
4,4,Carell,2022-04-04,None
5,4,Carell,2022-04-04,None
6,5,Joao,None,None
7,2,Braga,2022-02-02,None
8,3,Steve,2022-03-03,None
9,5,Joao,None,None


In [ ]:
spark.stop()